In [112]:
import os
import re

from transformers import pipeline
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

import spacy

import pandas as pd

extract name

In [113]:
#names
#names count

def extract_name(text):
  name_ls = []
  names_count = 0
  matches = re.findall(r'my name is (\w+) (\w+)', text, re.IGNORECASE)
  for match in matches:
    name_ls.append(match)
    names_count += 1
    
  return name_ls, names_count

extract keywords

In [114]:
#parking
#affordability
#community
#total words

def extract_keywords(text):
  words = text.split()
  words_count = len(words)

  word_hash = {}
  for word in words:
    if word not in word_hash:
        word_hash[word] = 1
    else:
        word_hash[word] += 1

  parking_count = 0
  if "parking" in word_hash:
     parking_count = word_hash["parking"]
  if "Parking" in word_hash:
     parking_count = word_hash["Parking"]
 

  affordable_count = 0
  if "affordability" in word_hash:
     affordable_count = word_hash["affordability"]
  if "affordable" in word_hash:
     affordable_count += word_hash["affordable"]
  if "Affordability" in word_hash:
     affordable_count = word_hash["Affordability"]
  if "Affordable" in word_hash:
     affordable_count += word_hash["Affordable"]
     

  community_count = 0
  if "community" in word_hash:
     community_count = word_hash["community"]
  if "communities" in word_hash:
     community_count += word_hash["communities"]
  if "Community" in word_hash:
     community_count = word_hash["Community"]
  if "Communities" in word_hash:
     community_count += word_hash["Communities"]



  
  return words_count, parking_count, affordable_count, community_count

sentiment

In [115]:
#sentiment
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"


tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

def get_sentiment(text):
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  return scores[0], scores[1], scores[2]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


word count

In [116]:
#text minus stop words
#text minus stop word count 

def remove_stop_words(text):
  words = text.split()
  non_stop_words_count = 0
  nlp = spacy.load("en_core_web_sm")
  stop_words = spacy.lang.en.stop_words.STOP_WORDS
  for i in range(len(words)):
     if words[i] not in stop_words:
        non_stop_words_count += 1
  return non_stop_words_count

merge

In [117]:
cleaned_hyperlinks_df = pd.read_csv("cleaned_hyperlinks2.csv")
case_to_df = {}

collapsed_df = cleaned_hyperlinks_df.groupby(['clean_file_name', 'case_name'])['audio_file_name'].agg(list).reset_index()


df = pd.DataFrame(columns=['case_name', 'CEQA', 'text', 'names', 'names_count', 'parking_mentions', 'affordability_mentions', 'community_mentions', 'num_words', 'num_nonstopgap_words', 'sentiment_pos', 'sentiment_neg', 'sentiment_neutral'])

In [119]:
#merge txt
#load in df
def get_ceqa(clean_file_name, case_name):
  filtered_rows = cleaned_hyperlinks_df[(cleaned_hyperlinks_df['clean_file_name'] == clean_file_name) & (cleaned_hyperlinks_df['case_name'] == case_name)]
  text_vals = filtered_rows['Text'].tolist()

  pattern = r"CEQA:\s*([\w-]+)"

  for text in text_vals:
    match = re.search(pattern, text)

    if match:
      code = match.group(1)
      print(code)
      return code
 
  
  return np.NaN


for index, row in collapsed_df.iterrows():
  file_names = row['audio_file_name']
  clean_file_name = row['clean_file_name']
  case_name = row['case_name']

  ceqa = get_ceqa(clean_file_name, case_name)
  text = " "
  for file in file_names:
    file_name = 'txt/' + file + '.txt'
    if os.path.exists(file_name):
      with open(file_name, 'r') as file:
        this_file_text = file.read()
        text += this_file_text
  print(file_name)
  name_ls, names_count = extract_name(text)
  words_count, parking_count, affordable_count, community_count = extract_keywords(text)
  if words_count > 500:
    negative, neutral, pos = get_sentiment(text[:1000]) #OR DO AVERAGE OF WHOLE TEXT?
  else:
    negative, neutral, pos = get_sentiment(text)
  non_stop_words_count = remove_stop_words(text)
      
  new_row = [case_name, ceqa, text, name_ls, names_count, parking_count, affordable_count, community_count, words_count, non_stop_words_count, pos, negative, neutral]
  #df = df.append(pd.Series(new_row, index=df.columns), ignore_index=True)
  df.loc[len(df)] = new_row


txt/01-25-2017audio_1 DirRpt.txt
txt/01-25-2017audio_2 CommBus.txt
txt/01-25-2017audio_3 NCP.txt
ENV-2016-1898-CE
txt/01-25-2017audio_4 VTT-74301+PubCom.txt
txt/07-12-2012audio_01DirectorsReport.txt
txt/07-12-2012audio_02CommissionBusiness.txt
txt/07-12-2012audio_03PublicCommentPeriod.txt
txt/07-12-2012audio_04CPC08-2142f.txt
ENV-2012-1505-CE
txt/07-12-2012audio_05CPC12-1504c.txt
txt/WLA 07-20-16audio_1 DepRpt.txt
txt/WLA 07-20-16audio_2 CommBus.txt
txt/WLA 07-20-16audio_3 NCP.txt
ENV-2015-1283-CE
txt/WLA 07-20-16audio_4 DIR-2015-1282-DRB.txt
ENV-2014-4445-MND
txt/WLA 07-20-16audio_5 ZA-2014-4444-CDP.txt
txt/WLA 07-20-16audio_6 PubCom.txt


In [120]:
df

,case_name,CEQA,text,names,names_count,parking_mentions,affordability_mentions,community_mentions,num_words,num_nonstopgap_words,sentiment_pos,sentiment_neg,sentiment_neutral
0,1 DirRp,NaN,"But this year ready. All right, good afterno...",[],0,0,0,0,305,178,0.542898,0.016691,0.440412
1,2 CommBu,NaN,So we have our next item of the agenda was c...,[],0,0,0,0,342,187,0.133361,0.017539,0.849100
2,3 NC,NaN,"On the agenda is number three, is the neighb...",[],0,0,0,0,39,21,0.180984,0.064153,0.754864
3,4 VTT-74301+PubCo,ENV-2016-1898-CE,,[],0,0,0,0,0,0,0.270926,0.058704,0.670370
4,01DirectorsRepor,NaN,Good morning everyone and welcome to a regul...,"[(Bill, Rochen)]",1,0,0,3,2739,1356,0.973028,0.001598,0.025374
5,02CommissionBusines,NaN,"Commissioners, can you check your calendars ...",[],0,5,0,1,4270,2106,0.363047,0.041718,0.595235
6,03PublicCommentPerio,NaN,"We do have one speaker, Mr. Herman.\n Can we...","[(Armando, Herman)]",1,0,0,0,390,208,0.291281,0.012802,0.695917
7,04CPC08-2142,NaN,PC 2008 2142 CA with the Associated Siqua. A...,"[(Rigoberto, Urtado), (Anna, C), (Isabel, Roja...",24,0,0,1,28822,14536,0.950415,0.002629,0.046956
8,05CPC12-1504,ENV-2012-1505-CE,Deeds will resume the hearing in just a few ...,"[(George, U), (King, Chung), (Tari, Statelyn),...",20,0,2,7,16639,8698,0.923370,0.003737,0.072893
9,1 DepRp,NaN,"Good afternoon everyone, we're going to call...","[(Harden, Carter)]",1,0,1,0,1402,714,0.183906,0.012833,0.803261


In [121]:
collapsed_df

,clean_file_name,case_name,audio_file_name
0,01-25-2017audio,1 DirRp,[01-25-2017audio_1 DirRpt]
1,01-25-2017audio,2 CommBu,[01-25-2017audio_2 CommBus]
2,01-25-2017audio,3 NC,[01-25-2017audio_3 NCP]
3,01-25-2017audio,4 VTT-74301+PubCo,[01-25-2017audio_4 VTT-74301+PubCom]
4,07-12-2012audio,01DirectorsRepor,[07-12-2012audio_01DirectorsReport]
5,07-12-2012audio,02CommissionBusines,[07-12-2012audio_02CommissionBusiness]
6,07-12-2012audio,03PublicCommentPerio,[07-12-2012audio_03PublicCommentPeriod]
7,07-12-2012audio,04CPC08-2142,"[07-12-2012audio_04CPC08-2142a, 07-12-2012audi..."
8,07-12-2012audio,05CPC12-1504,"[07-12-2012audio_05CPC12-1504a, 07-12-2012audi..."
9,WLA 07-20-16audio,1 DepRp,[WLA 07-20-16audio_1 DepRpt]
